In [2]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('r')
sc = SparkContext(conf = conf)

24/08/05 22:16:54 WARN Utils: Your hostname, MZC01-HYUCKSANGCHO.local resolves to a loopback address: 127.0.0.1; using 192.168.0.51 instead (on interface en0)
24/08/05 22:16:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/05 22:16:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 계좌번호, 거래금액
acTransList = ["SB10001,1000", "SB10002,1200", "SB10003,8000", "SB10004,400", "SB10005,300", "SB10006,10000", "SB10007,500", "SB10008,56", "SB10009,30","SB10010,7000", "CR10001,7000", "SB10002,-10"]
acTransRDD = sc.parallelize(acTransList)
acTransRDD.collect()

['SB10001,1000',
 'SB10002,1200',
 'SB10003,8000',
 'SB10004,400',
 'SB10005,300',
 'SB10006,10000',
 'SB10007,500',
 'SB10008,56',
 'SB10009,30',
 'SB10010,7000',
 'CR10001,7000',
 'SB10002,-10']

In [4]:
# 거래 금액이 0원 이상이고, 계좌번호가 SB로시작하는 데이터
from decimal import Decimal
goodTransRecords = acTransRDD.filter(lambda trans : Decimal(trans.split(',')[1]) > 0).filter(lambda trans : trans.split(',')[0].startswith('SB') == True)
goodTransRecords.collect()

['SB10001,1000',
 'SB10002,1200',
 'SB10003,8000',
 'SB10004,400',
 'SB10005,300',
 'SB10006,10000',
 'SB10007,500',
 'SB10008,56',
 'SB10009,30',
 'SB10010,7000']

In [5]:
# 거래 금액이 1000이상인 모든 고객
upper_1000_Recoard = acTransRDD.filter(lambda trans : Decimal(trans.split(',')[1]) >= 1000)
upper_1000_Recoard.collect()

['SB10001,1000',
 'SB10002,1200',
 'SB10003,8000',
 'SB10006,10000',
 'SB10010,7000',
 'CR10001,7000']

In [12]:
# 계좌번호 'SB'로 시작하지 않는 계좌만 출력
badAcNoLambda = acTransRDD.filter(lambda trans : trans.split(',')[0].startswith('SB') == False)
badAcNoLambda.collect()

['CR10001,7000']

In [23]:
# 모든 거래 금액의 합계
from decimal import Decimal
sumAmount = acTransRDD.map(lambda trans : Decimal(trans.split(',')[1])).reduce(lambda x,y : x+y)
sumAmount

Decimal('35476')

In [29]:
# 모든 거래 금액의 최댓값
maxAmount = acTransRDD.map(lambda trans : Decimal(trans.split(',')[1])).reduce(lambda x, y : x if x>y else y)
maxAmount

Decimal('10000')

In [30]:
# 모든 거래 금액의 최솟값
minAmount = acTransRDD.map(lambda trans : Decimal(trans.split(',')[1])).reduce(lambda x,y : x if x < y else y)
minAmount

Decimal('-10')

In [37]:
# 'SB'로 시작하지 않는 계좌만 출력
allGoodAccountNos = acTransRDD.flatMap(lambda trans : trans.split(',')).filter(lambda trans : trans.startswith('SB')==True)
allGoodAccountNos.collect()

['SB10001',
 'SB10002',
 'SB10003',
 'SB10004',
 'SB10005',
 'SB10006',
 'SB10007',
 'SB10008',
 'SB10009',
 'SB10010',
 'SB10002']

### ReduceByKey

In [39]:

from decimal import Decimal
# Creation of the list from where the RDD is going to be created
acTransList = ["SB10001,1000", "SB10002,1200", "SB10001,8000", "SB10002,400", "SB10003,300", "SB10001,10000", "SB10004,500", "SB10005,56", "SB10003,30","SB10002,7000", "SB10001,-100", "SB10002,-10"]
acTransRDD = sc.parallelize(acTransList)
acTransRDD.collect()

['SB10001,1000',
 'SB10002,1200',
 'SB10001,8000',
 'SB10002,400',
 'SB10003,300',
 'SB10001,10000',
 'SB10004,500',
 'SB10005,56',
 'SB10003,30',
 'SB10002,7000',
 'SB10001,-100',
 'SB10002,-10']

In [40]:
acKeyVal = acTransRDD.map(lambda trans : (trans.split(',')[0], Decimal(trans.split(',')[1])))
acKeyVal.collect()

[('SB10001', Decimal('1000')),
 ('SB10002', Decimal('1200')),
 ('SB10001', Decimal('8000')),
 ('SB10002', Decimal('400')),
 ('SB10003', Decimal('300')),
 ('SB10001', Decimal('10000')),
 ('SB10004', Decimal('500')),
 ('SB10005', Decimal('56')),
 ('SB10003', Decimal('30')),
 ('SB10002', Decimal('7000')),
 ('SB10001', Decimal('-100')),
 ('SB10002', Decimal('-10'))]

In [42]:
accSummary = acKeyVal.reduceByKey(lambda x, y : x+y).sortByKey()
accSummary.collect()

[('SB10001', Decimal('18900')),
 ('SB10002', Decimal('8590')),
 ('SB10003', Decimal('330')),
 ('SB10004', Decimal('500')),
 ('SB10005', Decimal('56'))]

### Join App

In [43]:
# Creation of the list from where the RDD is going to be created
acMasterList = ["SB10001,Roger,Federer", "SB10002,Pete,Sampras", "SB10003,Rafel,Nadal", "SB10004,Boris,Becker", "SB10005,Ivan,Lendl"]
# Creation of the list from where the RDD is going to be created
acBalList = ["SB10001,50000", "SB10002,12000", "SB10003,3000", "SB10004,8500", "SB10005,5000"]
# Create the RDD
acMasterRDD = sc.parallelize(acMasterList)
# Create the RDD
acBalRDD = sc.parallelize(acBalList)

In [45]:
acMasterRDD.collect()

['SB10001,Roger,Federer',
 'SB10002,Pete,Sampras',
 'SB10003,Rafel,Nadal',
 'SB10004,Boris,Becker',
 'SB10005,Ivan,Lendl']

In [46]:
acBalRDD.collect()

['SB10001,50000',
 'SB10002,12000',
 'SB10003,3000',
 'SB10004,8500',
 'SB10005,5000']

In [51]:
acMasterTuples = acMasterRDD.map(lambda trans : trans.split(',')).map(lambda trans : (trans[0], trans[1] + ' ' + trans[2]))
acMasterTuples.collect()

[('SB10001', 'Roger Federer'),
 ('SB10002', 'Pete Sampras'),
 ('SB10003', 'Rafel Nadal'),
 ('SB10004', 'Boris Becker'),
 ('SB10005', 'Ivan Lendl')]

In [57]:
acBalTuples = acBalRDD.map(lambda trans : trans.split(',')).map(lambda transList : (transList[0], transList[1]))
acBalTuples.collect()

[('SB10001', '50000'),
 ('SB10002', '12000'),
 ('SB10003', '3000'),
 ('SB10004', '8500'),
 ('SB10005', '5000')]

In [65]:
acJoinTuples = acMasterTuples.join(acBalTuples).sortByKey().map(lambda tran : (tran[0], tran[1][0], tran[1][1]))
acJoinTuples.collect()

[('SB10001', 'Roger Federer', '50000'),
 ('SB10002', 'Pete Sampras', '12000'),
 ('SB10003', 'Rafel Nadal', '3000'),
 ('SB10004', 'Boris Becker', '8500'),
 ('SB10005', 'Ivan Lendl', '5000')]

In [66]:
# Account Name과 Balance 찾기
acNameAndBalance = acJoinTuples.map(lambda trans : (trans[1], trans[2]))
acNameAndBalance.collect()

[('Roger Federer', '50000'),
 ('Pete Sampras', '12000'),
 ('Rafel Nadal', '3000'),
 ('Boris Becker', '8500'),
 ('Ivan Lendl', '5000')]

In [68]:
# Amout가 가장 큰 element 찾기
from decimal import Decimal
acTuplesByAmount = acNameAndBalance.map(lambda tran: (Decimal(tran[1]), tran[0])).sortByKey(False)
acTuplesByAmount.first()

(Decimal('50000'), 'Roger Federer')

In [69]:
# top3
acTuplesByAmount.take(3)

[(Decimal('50000'), 'Roger Federer'),
 (Decimal('12000'), 'Pete Sampras'),
 (Decimal('8500'), 'Boris Becker')]

In [70]:
# Count by keys
acBalTuples.countByKey()

defaultdict(int,
            {'SB10001': 1,
             'SB10002': 1,
             'SB10003': 1,
             'SB10004': 1,
             'SB10005': 1})

In [71]:
# Count all the records
acBalTuples.count()

5

In [72]:
acNameAndBalance.foreach(print)

('Roger Federer', '50000')
('Pete Sampras', '12000')
('Rafel Nadal', '3000')
('Boris Becker', '8500')
('Ivan Lendl', '5000')
